# pyaedt AnsysLAB proof of concept

Uses pypim to create an aedt instance, and then uses pyaedt to 
prepare an HFSS project, solve it and display its solution

### Assumptions:
- client files must be available in ansysedt_client_basedir
- syslib files must be available in ansysedt_client_basedir/syslib 
- userlib files must be available in ansysedt_client_basedir/userlib


### Known issues: 
- HFSS analysis setup with a sweep fails to solve due to MPI configuration issue
- Hardcodes ansysedt_client_basedir  to '/home/jovyan/shared/amathur/client_files' 



## Install PyPim and client files for AEDT scripting

In [1]:
#First install pypim
#!pip install ansys-platform-instancemanagement
#!pip install pyaedt

In [2]:
# Next install the binaries for ansysedt client if needed
# %cd ~/shared/amathur
#!tar xf ansysedt_python_client.tar.gz
#!ls /home/jovyan/shared/amathur/client_files

# Example - POC

This POC uses  a non-standard (and non-pythonic) way of installing the client files needed for aedt scripting environment

Instead, Alex has suggested preparing a python wheel. 

## Using PyPim, create an aedt instance 

In [3]:
import ansys.platform.instancemanagement as pypim
import os

# temporary: target the PIM with AEDT configured
# os.environ["ANSYS_PLATFORM_INSTANCEMANAGEMENT_CONFIG"]="/home/jovyan/shared/plule/pypim-aedt.json"

pim = pypim.connect()
pim.list_definitions()
aedt = pim.create_instance(product_name="aedt")
aedt.wait_for_ready()

# Workaround, retrieve the ip and port from the address, this will break in the future
address = aedt.services["grpc"].uri.split(":")
port = int(address[-1])
machine = address[-2]
print(machine, port)

10.240.0.191 50051


## Add the location of DesktopPlugin python files to system path

In [4]:
import sys
import os
# Change the ansysedt_client_basedir path as needed
ansysedt_client_basedir='/home/jovyan/shared/amathur/client_files'
# Add the location of DesktopPlugin python files to system path
sys.path.append(os.path.join(ansysedt_client_basedir,'PythonFiles/DesktopPlugin'))


## Initialize aedt scripting environment

In [5]:
import ScriptEnv

ScriptEnv.Initialize("Ansoft.ElectronicsDesktop",machine=machine,port=port,NG=True)

E0616 22:01:26.317181133    1069 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
/home/jovyan/shared/amathur/client_files/PythonFiles/DesktopPlugin/ScriptEnv.py:115: UserWarning: Application was launched from different directory
  warnings.warn("Application was launched from different directory")


## Import PyAedt


In [6]:
import pyaedt
from pyaedt import Hfss
from pyaedt import settings


## Configuration
ANSYSEM_LOCAL_SYSLIB needs to point to a valid syslib that is available locally

In [7]:
## Environment variables
os.environ['ANSYSEM_LOCAL_SYSLIB'] = os.path.join(ansysedt_client_basedir,"syslib")
os.environ['ANSYSEM_LOCAL_USERLIB'] = os.path.join(ansysedt_client_basedir,"userlib")

pyaedt configuration for AnsysLab python environment that is needed for using the oDesktop object that is already initialized 

In [8]:
# Initialize pyaedt's logging and temp directory 
import tempfile
pyaedt.desktop._com = "python3"
tempfile.tempdir = "/tmp"
settings.logger_file_path="/tmp/pyaedt.log"
oDesktop.SetProjectDirectory('/tmp')

In [9]:
desktop = pyaedt.desktop.Desktop()

Logger Started on /tmp/pyaedt.log
pyaedt v0.4.34
Python version 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) 
[GCC 10.3.0]


pyaedt info: Logger Started on /tmp/pyaedt.log
pyaedt info: pyaedt v0.4.34
pyaedt info: Python version 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) 
[GCC 10.3.0]


## Define HfssProject class to override syslib etc
pyaedt needs locally available syslib directory

In [10]:
# Assumes that environment variables ANSYSEM_LOCAL_SYSLIB and ANSYSEM_LOCAL_USERLIB point to locally available syslib and userlib resp.

class HfssProject(Hfss,object):
    def __init__(
        self,
        projectname=None,
        designname=None,
        solution_type=None,
        setup_name=None,
        specified_version=None,
        non_graphical=False,
        new_desktop_session=False,
        close_on_exit=False,
        student_version=False,
    ):
        Hfss.__init__(
            self,
            projectname,
            designname,
            solution_type,
            setup_name,
            specified_version,
            non_graphical,
            new_desktop_session,
            close_on_exit,
            student_version,
        )
        self.field_setups = self._get_rad_fields()

    def _local_lib(self,lib_name):
        """Sys/User/PersonalLib directory on the local host using the value of ANSYSEM_LOCAL_xxx environment variable.
        Parameters
        ----------
        lib_name : str
            Library name. ``'syslib','userlib','personallib'``.

        Returns
        -------
        str
            Full absolute path for the specified directory on the local host.

        """
        if not lib_name in "syslib,userlib,personallib":
            return ""
        env_name = "ANSYSEM_LOCAL_%s" % lib_name.upper()
        local_path = os.environ.get(env_name,"")
        if local_path and os.path.exists(local_path):
            return local_path
        return ""
    
    def _locally_available_lib(self,desktop_lib_path,lib_name):
        """returns a locally available path for lib_name. If the path provided by desktop is not available locally, it return 
        
        Parameters
        ----------
        lib_name : str
            Library name. ``'syslib','userlib','personallib'``.

        Returns
        -------
        str
            Full absolute path for the specified directory that is available locally.
            Returns an empty string if the library is not available locally

        """
        if os.path.exists(desktop_lib_path):
            return os.path.normpath(desktop_lib_path)
        local_path = self._local_lib(lib_name)
        if local_path:
            local_path = os.path.normpath(local_path)
            ## self.logger.info("HfssProject: Library %s switched locally from %s to %s.", lib_name, desktop_lib_path, local_path)
        return local_path
        
    @property
    def personallib(self):
        """PersonalLib directory.

        Returns
        -------
        str
            Full absolute path for the ``PersonalLib`` directory.

        References
        ----------

        >>> oDesktop.GetPersonalLibDirectory
        """
        return self._locally_available_lib(self.odesktop.GetPersonalLibDirectory(),"personallib")

    @property
    def userlib(self):
        """UserLib directory.

        Returns
        -------
        str
            Full absolute path for the ``UserLib`` directory.

        References
        ----------

        >>> oDesktop.GetUserLibDirectory
        """
        return self._locally_available_lib(self.odesktop.GetUserLibDirectory(),"userlib")

    @property
    def syslib(self):
        """SysLib directory.

        Returns
        -------
        str
            Full absolute path for the ``SysLib`` directory.

        References
        ----------

        >>> oDesktop.GetLibraryDirectory
        """
        return self._locally_available_lib(self.odesktop.GetLibraryDirectory(),"syslib")  
    
    def get_remote_syslib_path(self,filepath):
        """Replaces SysLib directory to the directory on the remote server.

        Returns
        -------
        str
            Full absolute path for the file in ``SysLib`` directory on the remote server.

        """
        return filepath.replace(self.syslib,self.odesktop.GetLibraryDirectory())        
        

## Run a pyaedt example 
Based on pyaedt's HFSS_Dipole example

Note that instead of pyaedt's Hfss class, it uses the above HfssProject class, which is a derived class

In [11]:
hfss = HfssProject(solution_type="Modal")


Project Project1 has been created.
No design is present. Inserting a new design.


pyaedt info: Project Project1 has been created.
pyaedt info: No design is present. Inserting a new design.


Design Loaded
Successfully loaded project materials !
Materials Loaded


pyaedt info: Design Loaded
pyaedt info: Successfully loaded project materials !
pyaedt info: Materials Loaded


In [12]:
"Library directory in the AnsysEM container: %s" % hfss.odesktop.GetLibraryDirectory()

'Library directory in the AnsysEM container: /opt/AnsysEM/v222/Linux64/syslib'

In [13]:
"Library directory available locally: %s" % hfss.syslib

'Library directory available locally: /home/jovyan/shared/amathur/client_files/syslib'

In [14]:
# From pyaedt/examples/02-HFSS/HFSS_Dipole.py
###############################################################################
# Define a Dipole Length Variable
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# This command defines a dipole length variable.

hfss["l_dipole"] = "13.5cm"

###############################################################################
# Get a 3D Component from the `syslib` Directory
# ----------------------------------------------
# To run this example correctly, you must get all geometry parameters of the
# 3D component or, in case of an encrypted 3D component, create a dictionary
# of the parameters.

compfile = hfss.components3d["Dipole_Antenna_DM"]

geometryparams = hfss.get_components3d_vars("Dipole_Antenna_DM")
geometryparams["dipole_length"] = "l_dipole"
compfile = hfss.get_remote_syslib_path(compfile)
hfss.modeler.insert_3d_component(compfile, geometryparams)

'Dipole_Antenna1'

In [15]:

###############################################################################
# Create the Setup
# ----------------
# A setup with a sweep will be used to run the simulation.

setup = hfss.create_setup("MySetup")
setup.props["Frequency"] = "1GHz"
setup.props["MaximumPasses"] = 4
setup.update()


True

In [ ]:
# Known issue: the sweep creation always causes analysis to fail due to MPI configuration issue
hfss.create_linear_count_sweep(
    setupname=setup.name,
    unit="GHz",
    freqstart=0.5,
    freqstop=1.5,
    num_of_freq_points=251,
    sweepname="sweep1",
    sweep_type="Interpolating",
    interpolation_tol=3,
    interpolation_max_solutions=255,
    save_fields=False,
)


In [16]:
###############################################################################
# Create Boundaries
# -----------------
# A region with openings is needed to run the analysis.

hfss.create_open_region(Frequency="1GHz")

Open Region correctly created.


pyaedt info: Open Region correctly created.


True

In [17]:
hfss.save_project(os.path.join("/tmp", "MyDipole-example.aedt"))



Saving Project1 Project


pyaedt info: Saving Project1 Project


True

In [18]:
hfss.analyze_setup("MySetup")

Solving design setup MySetup
Design setup MySetup solved correctly


pyaedt info: Solving design setup MySetup
pyaedt info: Design setup MySetup solved correctly


True

In [23]:
hfss.odesktop.GetMessages("","",0)

['GRPC server running on port: 50051.\n',
 'Logger Started on /tmp/pyaedt.log',
 'pyaedt v0.4.34',
 'Python version 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) \n[GCC 10.3.0]',
 'Project Project1 has been created.',
 'No design is present. Inserting a new design.',
 'Design Loaded',
 'Successfully loaded project materials !',
 'Materials Loaded',
 'Open Region correctly created.',
 'Saving Project1 Project',
 'Solving design setup MySetup',
 'Design setup MySetup solved correctly',
 'Solution Data Correctly Loaded.',
 'Adaptive Passes did not converge based on specified criteria. (10:04:20 PM  Jun 16, 2022)',
 'Normal completion of simulation on server: Local Machine. (10:04:20 PM  Jun 16, 2022)',
 'Script macro error: No data found for 1GHz (10:04:54 PM  Jun 16, 2022)']

In [ ]:
###############################################################################
# Postprocessing
# --------------
# Generate a scattering plot and a far fields plot.

# hfss.create_scattering("MyScattering")
variations = hfss.available_variations.nominal_w_values_dict
variations["Freq"] = ["1GHz"]
variations["Theta"] = ["All"]
variations["Phi"] = ["All"]
hfss.post.create_rectangular_plot(
    "db(GainTotal)", hfss.nominal_adaptive, variations, "Theta", "3D", report_category="Far Fields"
)


In [ ]:
###############################################################################
# Plot the model
# ~~~~~~~~~~~~~~

my_plot = hfss.plot(show=False, plot_air_objects=False)
my_plot.show_axes = False
my_plot.show_grid = False
my_plot.isometric_view = False
my_plot.plot(
    os.path.join("/tmp", "Image.jpg"),
)

In [21]:
variations = hfss.available_variations.nominal_w_values_dict
variations["Freq"] = ["1GHz"]
variations["Theta"] = ["All"]
variations["Phi"] = ["All"]


In [22]:
solutions = hfss.post.get_solution_data_per_variation(

    "GainTotal",

    hfss.nominal_adaptive,

    variations,

#    primary_sweep_variable="Theta",

#    context="3D",

#    report_category="Far Fields",

)

Solution Data Correctly Loaded.


pyaedt info: Solution Data Correctly Loaded.


  File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 601, in run_forever
    self._run_once()
  File "/opt/conda/lib/python3.9/asyncio/base_events.py", line 1905, in _run_once
    handle._run()
  File "/opt/conda/lib/python3.9/asyncio/events.py", line 80, in _run
    self._co

In [24]:
 #hfss.nominal_adaptive, variations, 
 solutions.data_real()

[1.0]

## Stop aedt instance

In [25]:
aedt.delete()